In [ ]:
import numpy as np
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

# Initialize IBM Runtime Service and Backend
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Step 1: Map classical inputs of the Lorenz attractor to a quantum problem

# Define parameters corresponding to classical system variables
x = Parameter("x")
y = Parameter("y")
z = Parameter("z")

# Create a quantum circuit to encode the Lorenz system
lorenz_circuit = QuantumCircuit(3)
lorenz_circuit.h(0)  # Initialize in a superposition state
lorenz_circuit.h(1)
lorenz_circuit.h(2)

# Parameterized rotations to encode x, y, z as parameters
lorenz_circuit.rx(x, 0)
lorenz_circuit.ry(y, 1)
lorenz_circuit.rz(z, 2)

# Step 2: Define Observables (corresponding to operators in the Lorenz system)
ZZI = SparsePauliOp.from_list([("ZZI", 1)])
ZIZ = SparsePauliOp.from_list([("ZIZ", 1)])
IZZ = SparsePauliOp.from_list([("IZZ", 1)])
XXX = SparsePauliOp.from_list([("XXX", 1)])
ZZZ = SparsePauliOp.from_list([("ZZZ", 1)])
ops = [ZZI, ZIZ, IZZ, XXX, ZZZ]

# Step 3: Prepare for optimization and quantum execution

# Optimize the circuit layout for the backend
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
optimized_circuit = pm.run(lorenz_circuit)
isa_observables = [
    operator.apply_layout(optimized_circuit.layout) for operator in ops
]

# Define parameterized values
number_of_steps = 21
x_vals = np.linspace(0, 2 * np.pi, number_of_steps)
y_vals = np.linspace(0, 2 * np.pi, number_of_steps)
z_vals = np.linspace(0, 2 * np.pi, number_of_steps)

parameter_sets = [[x_val, y_val, z_val] for x_val, y_val, z_val in zip(x_vals, y_vals, z_vals)]

# Step 4: Execute using Qiskit Estimator

# Reshape observable array for broadcasting
reshaped_ops = np.fromiter(isa_observables, dtype=object)
reshaped_ops = reshaped_ops.reshape((5, 1))

# Initialize the Estimator
estimator = Estimator(backend, options={"default_shots": int(1e4)})

# Run the parameterized experiment
job = estimator.run([(optimized_circuit, reshaped_ops, parameter_sets)])

# Get results
result = job.result()[0]
print(f">>> Expectation values: {result.data.evs}")
print(f">>> Standard errors: {result.data.stds}")
print(f">>> Metadata: {result.metadata}")